# **ANALIZAR RESPUESTAS**

# 1. Autenticación
Este paso de autenticación es necesario para que Colab pueda acceder al Sheet que tiene las respuestas. Se abrirán dos ventanas y tendrás que aceptar los permisos para continuar.

In [ ]:
# Conecta tu unidad de Google Drive.
# Se abrirá otra ventana. Aceptá los permisos para continuar.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Autentica al usuario en Google Colab y configura las credenciales para acceder a Google Sheets.
# Se abrirá otra ventana. Aceptá los permisos para continuar.
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Abre y lee el Sheet que tiene las respuestas para analizar.
# Este Sheet debe estar en tu unidad de Drive y debe tener una sola columna con una respuesta por fila.

# El nombre del Sheet debe ser "Respuestas" (o si querés ponerle otro nombre, asegurate de cambiar el nombre en la siguiente línea de código).
# Debe tener dos hojas, "respuestas_es" con las respuestas en español y "respuestas_pt" con las respuestas en portugués.

sheet = gc.open("Respuestas")

# Lee las respuestas en español.
f1 = sheet.get_worksheet(0)
respuestas_es = f1.get_all_values()

# Lee las respuestas en portugués.
f2 = sheet.get_worksheet(1)
respuestas_pt = f2.get_all_values()

In [ ]:
# Imprime la lista de las respuestas.
print(respuestas_es)
print(respuestas_pt)

# 2. Clasificación de texto
Esta parte del código utiliza el clasificador para categorizar las respuestas en español por tema (producto, servicio, general). Este clasificador no funciona con respuestas en portugués.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
# Carga el clasificador entrenado en las respuestas a las encuestas anteriores.
# Asegurate de que el clasificador y el tokenizador no estén en ninguna carpeta en Drive.
# Si preferís que permanezcan en una carpeta, asegurate de cambiar las rutas en las siguientes líneas de código.
loaded_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/model_sinprecio2")

# Carga el tokenizador pre-entrenado.
loaded_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/model_sinprecio2")

In [ ]:
# Crea un diccionario para convertir los IDs a etiquetas.
# El modelo que entrené le asignó el 0 a "general", el 1 a "producto" y el 2 a "servicio".
id2label = {0: "general", 1: "producto", 2: "servicio"}

# Hace una lista para guardar los resultados del análisis de las respuestas en español.
resultados_es = []

In [ ]:
# En esta sección, el clasificador hace predicciones.

# Para cada respuesta...
for respuesta in respuestas_es:
    inputs = loaded_tokenizer(respuesta[0], return_tensors="pt")
    outputs = loaded_model(**inputs)
    # Hace una predicción.
    clase = outputs.logits.argmax(dim=-1).item()
    # Agrega la predicción a la lista de resultados.
    resultados_es.append([respuesta[0], id2label[clase]])

In [ ]:
# Imprime la lista de resultados actualizada.

# Ahora tenemos una lista de las respuestas en español y su categoría.
print(resultados_es)

# 3. Análisis de sentimiento
Esta parte del código utiliza dos modelos de análisis de sentimiento entrenados con tweets en español y portugués ([pysentimiento](https://github.com/pysentimiento/pysentimiento/)) para analizar el sentimiento de las respuestas en ambos idiomas.

In [ ]:
# Este paso puede tardar un poco.
!pip install pysentimiento transformers accelerate

In [ ]:
sentimientos = {"NEG": "negativo", "POS": "positivo", "NEU": "neutro"}

In [ ]:
from pysentimiento import create_analyzer

# Crea un analyzer con la tarea de análisis de sentimiento en español.
analyzer_es = create_analyzer(task="sentiment", lang="es")

# Hace predicciones de sentimiento y las guarda en la lista de resultados.
for i in range(len(resultados_es)):
    sentimiento = analyzer_es.predict(resultados_es[i][0])
    resultados_es[i].append(sentimientos[sentimiento.output])

In [ ]:
# Hace una lista para guardar los resultados del análisis de las respuestas en portugués.
resultados_pt = [x for x in respuestas_pt]

# Crea un analyzer con la tarea de análisis de sentimiento en portugués.
analyzer_pt = create_analyzer(task="sentiment", lang="pt")

# Hace predicciones de sentimiento y las guarda en la lista de resultados.
for i in range(len(resultados_pt)):
    sentimiento = analyzer_pt.predict(resultados_pt[i][0])
    resultados_pt[i].append(sentimientos[sentimiento.output])

In [ ]:
# Imprime las listas de resultados actualizadas.

# Ahora tenemos una lista con las respuestas en español, su categoría y su sentimiento.
print(resultados_es)

# Y tenemos otra lista con las respuestas en portugués y su sentimiento.
print(resultados_pt)

# 4. Extracción de palabras clave

Esta parte del programa extrae los sustantivos más importantes de cada respuesta utilizando [este paquete de spaCy](https://spacy.io/models/es) para las respuestas en español y [este paquete](https://spacy.io/models/pt) para las respuestas en portugués.

Para hacerlo, usé como referencia [este artículo sobre el modelado temático con texto en español](https://medium.com/@jarch/topic-modeling-with-spanish-text-an-introductory-example-in-python-9cdb10fbe126), [este artículo sobre el análisis de texto](https://medium.com/nlplanet/text-analysis-topic-modelling-with-spacy-gensim-4cd92ef06e06#3363) y [este artículo sobre spaCy y gensim](https://towardsdatascience.com/topic-modelling-in-python-with-spacy-and-gensim-dc8f7748bdbf).



In [ ]:
# Descarga el paquete de "es_core_news_sm" de spaCy.
# Este paso puede tardar un poco.
!python -m spacy download es_core_news_sm

In [ ]:
import nltk
from nltk.corpus import stopwords as nltk_stopwords
import spacy

# Descarga una lista de stopwords, o sea, palabras funcionales y palabras frecuentes en español (ej. "de", "los", "que").
# Estas palabras las vamos a eliminar para obtener una lista de palabras clave.
nltk.download("stopwords")
spanish_stopwords = set(nltk_stopwords.words("spanish"))

# Agrega unos stopwords personalizados, propios de este proyecto.
# Por ejemplo, muchas respuestas mencionan Loyal Solutions pero no queremos que se agreguen esas palabras a la lista de palabras clave.
es_custom_stopwords = ["loyal", "solutions", "vez", "veces", "cosa", "año", "usamos"]

for w in es_custom_stopwords:
    spanish_stopwords.add(w)

print(spanish_stopwords)

In [ ]:
# Carga el paquete de NLP en español.
nlp = spacy.load("es_core_news_sm")

# Do something to make sure words don't repeat? FIXMEEE

# Define una función para normalizar, tokenizar y lematizar las respuestas en español.
# Esta función recibe una respuesta, limpia la respuesta y genera una lista de palabras clave.
def es_normalize_tokenize_lemmatize(text):
    doc = nlp(text)
    VALID_POS = {'PROPN', 'NOUN'}
    keywords = []
    for word in doc:
        if word.lemma_ and word.lemma_.lower() not in spanish_stopwords and word.lemma_.lower() not in keywords:
            if word.pos_ in VALID_POS:
                keywords.append(word.lemma_.lower())
    return keywords

# Actualiza la lista de resultados en español.
for resultado in resultados_es:
    resultado.append(", ".join(es_normalize_tokenize_lemmatize(resultado[0])))

In [ ]:
# Descarga el paquete de "pt_core_news_sm" de spaCy.
# Este paso puede tardar un poco.
!python -m spacy download pt_core_news_sm

In [ ]:
# Ahora, repetimos el proceso de arriba con las respuestas en portugués.

# Descarga una lista de stopwords, o sea, palabras funcionales y palabras frecuentes en portugués (ej. "de", "os", "que").
# Estas palabras las vamos a eliminar para obtener una lista de palabras clave.
portuguese_stopwords = set(nltk_stopwords.words("portuguese"))

# Agrega unos stopwords personalizados, propios de este proyecto.
# Por ejemplo, muchas respuestas mencionan las palabras Loyal y Solutions pero no queremos que aparezcan a la lista de palabras clave.
pt_custom_stopwords = ["loyal", "solutions", "vez", "vezes"]

for w in pt_custom_stopwords:
    portuguese_stopwords.add(w)

print(portuguese_stopwords)

In [ ]:
# Carga el paquete de NLP en portugués.
nlp2 = spacy.load("pt_core_news_sm")

# Define una función para normalizar, tokenizar y lematizar las respuestas en portugués.
# Esta función recibe una respuesta, limpia la respuesta y genera una lista de palabras clave.
def pt_normalize_tokenize_lemmatize(text):
    doc = nlp2(text)
    VALID_POS = {'PROPN', 'NOUN'}
    keywords = []
    for word in doc:
        if word.lemma_ and word.lemma_.lower() not in portuguese_stopwords and word.lemma_.lower() not in keywords:
            if word.pos_ in VALID_POS:
                keywords.append(word.lemma_.lower())
    return keywords

# Actualiza la lista de resultados en portugués.
for resultado in resultados_pt:
    resultado.append(", ".join(pt_normalize_tokenize_lemmatize(resultado[0])))

In [ ]:
# Imprime los resultados finales.
print(resultados_es)
print(resultados_pt)

# 5. Combinar y analizar

In [ ]:
# Inicia una lista de todos los resultados.
resultados_todo = [["Respuesta", "Idioma", "Categoría", "Sentimiento", "Palabras clave"]]

# Combina los resultados de las respuestas en español y los de las respuestas en portugués.
for i in range(len(resultados_es)):
    resultados_todo.append([resultados_es[i][0], "español", resultados_es[i][1], resultados_es[i][2], resultados_es[i][3]])
for i in range(len(resultados_pt)):
    resultados_todo.append([resultados_pt[i][0], "portugués", "", resultados_pt[i][1], resultados_pt[i][2]])

In [ ]:
import pandas as pd

# Convierte la lista de todos los resultados en Dataframe.
resultados_df = pd.DataFrame(resultados_todo)

In [ ]:
# Realiza un análisis básico de todos los resultados.

analisis = [["Categoría", "Cantidad de respuestas"]]

num_es = resultados_df[resultados_df[1] == "español"].shape[0]
num_pt = resultados_df[resultados_df[1] == "portugués"].shape[0]
analisis.append(["Respuestas en español", num_es])
analisis.append(["Respuestas en portugués", num_pt])

# Categoría
num_general = resultados_df[resultados_df[2] == "general"].shape[0]
num_producto = resultados_df[resultados_df[2] == "producto"].shape[0]
num_servicio = resultados_df[resultados_df[2] == "servicio"].shape[0]
analisis.append(["Respuestas sobre el producto", num_producto])
analisis.append(["Respuestas sobre el servicio", num_servicio])
analisis.append(["Respuestas generales", num_general])

# Sentimiento
num_negativo = resultados_df[resultados_df[3] == "negativo"].shape[0]
num_positivo = resultados_df[resultados_df[3] == "positivo"].shape[0]
num_neutro = resultados_df[resultados_df[3] == "neutro"].shape[0]
analisis.append(["Respuestas positivas", num_positivo])
analisis.append(["Respuestas negativas", num_negativo])
analisis.append(["Respuestas neutras", num_neutro])

# Categoría y sentimiento
pos_prod = resultados_df[resultados_df[2] == "producto"][resultados_df[3] == "positivo"].shape[0]
neg_prod = resultados_df[resultados_df[2] == "producto"][resultados_df[3] == "negativo"].shape[0]
neu_prod = resultados_df[resultados_df[2] == "producto"][resultados_df[3] == "neutro"].shape[0]
pos_serv = resultados_df[resultados_df[2] == "servicio"][resultados_df[3] == "positivo"].shape[0]
neg_serv = resultados_df[resultados_df[2] == "servicio"][resultados_df[3] == "negativo"].shape[0]
neu_serv = resultados_df[resultados_df[2] == "servicio"][resultados_df[3] == "neutro"].shape[0]
pos_gen = resultados_df[resultados_df[2] == "general"][resultados_df[3] == "positivo"].shape[0]
neg_gen = resultados_df[resultados_df[2] == "general"][resultados_df[3] == "negativo"].shape[0]
neu_gen = resultados_df[resultados_df[2] == "general"][resultados_df[3] == "neutro"].shape[0]

analisis.append(["Respuestas positivas sobre el producto", pos_prod])
analisis.append(["Respuestas positivas sobre el servicio", pos_serv])
analisis.append(["Respuestas positivas generales", pos_gen])

analisis.append(["Respuestas negativas sobre el producto", neg_prod])
analisis.append(["Respuestas negativas sobre el servicio", neg_serv])
analisis.append(["Respuestas negativas generales", neg_gen])

analisis.append(["Respuestas neutras sobre el producto", neu_prod])
analisis.append(["Respuestas neutras sobre el servicio", neu_serv])
analisis.append(["Respuestas neutras generales", neu_gen])

In [ ]:
print(analisis)

In [ ]:
# Convierte la lista de análisis en Dataframe.
analisis_df = pd.DataFrame(analisis[1:], columns=analisis[0])
analisis_df.head(20)

In [ ]:
# Palabras clave
from collections import Counter

palabras_clave = []
for i in range(len(resultados_todo)):
    palabras_clave += resultados_todo[i][4].split(", ")
palabras_dict = Counter(palabras_clave)

print(palabras_dict)

palabras = [["Palabra", "Frecuencia"]]

for k, v in palabras_dict.items():
    if k != "" and v > 3:
        palabras.append([k, v])

print(palabras)

In [ ]:
# Convierte el diccionario de palabras clave en Dataframe.
palabras_df = pd.DataFrame(palabras[1:], columns=palabras[0])
palabras_df.head(20)

# 6. Exportar

In [ ]:
# Crea cinco nuevas hojas del Sheet llamadas "resultados_es", "resultados_pt", "resultados_todo", "análisis" y "palabras_clave"
# y escribe los datos en el Sheet.
spreadsheet = gc.open("Respuestas")

# resultados_es tiene los resultados de las respuestas en español.
sheet_es = spreadsheet.add_worksheet(title="resultados_es", rows=str(len(resultados_es)+2), cols=str(len(resultados_es[0])+2))
sheet_es.update("A1", [["Respuesta", "Categoría", "Sentimiento", "Palabras clave"]] + resultados_es)

# resultados_pt tiene los resultados de las respuestas en portugués.
sheet_pt = spreadsheet.add_worksheet(title="resultados_pt", rows=str(len(resultados_pt)+2), cols=str(len(resultados_pt[0])+2))
sheet_pt.update("A1", [["Respuesta", "Sentimiento", "Palabras clave"]] + resultados_pt)

# resultados_todo tiene los resultados de todas las respuestas en español y portugués.
sheet_todo = spreadsheet.add_worksheet(title="resultados_todo", rows=str(len(resultados_todo)+2), cols=str(len(resultados_todo[0])+2))
sheet_todo.update("A1", resultados_todo)

# análisis tiene las cantidades de respuestas en diferentes categorías.
sheet_analisis = spreadsheet.add_worksheet(title="análisis", rows=str(len(analisis)+2), cols=str(len(analisis[0])+2))
sheet_analisis.update("A1", analisis)

# palabras_clave tiene una lista de las palabras clave y sus frecuencias correspondientes.
sheet_palabras = spreadsheet.add_worksheet(title="palabras_clave", rows=str(len(palabras)+2), cols=str(len(palabras[0])+2))
sheet_palabras.update("A1", palabras)